Find Candiate molecule through FP match


In [ ]:
from rdkit import Chem
from rdkit.DataStructs import FingerprintSimilarity as tanimoto
from rdkit.Chem import Draw
from rdkit.Chem import AllChem,MACCSkeys
import pandas as pd
from ast import literal_eval
from joblib import Parallel, delayed

In [ ]:
class SimilarityFP():
    def __init__(self):
        pass
    def Similarity(self,smiles,target):
        #smiles: SMILES of the molecule
        # target : MACCS key of target moelcule
        try:
            fp=MACCSkeys.GenMACCSKeys(Chem.MolFromSmiles(smiles))
            score=tanimoto(fp,target)
        except:
            score=0

        return score
    def FP_score(self, molecules,target,path):
        '''
        molecules : List of molecules to compare against
        target : target molecule to compared with (SMARTS/SMILES string)
        path : Path (including filename) to store list of similar molecules
        '''
        targetmol=Chem.MolFromSmarts(target)
        Chem.SanitizeMol(targetmol)
        target=MACCSkeys.GenMACCSKeys(targetmol)
        score=Parallel(n_jobs=-1,verbose=1)(delayed(self.Similarity)(smiles,target) for smiles in molecules)
        pd.DataFrame({'Molecules':molecules,'Similarity':score}).sort_values(by='Similarity', ascending=False).to_csv(path)
        

In [ ]:
# Example implementation

df=pd.read_csv('FinalSmiles.csv',header=0, low_memory = False) # File containing SMILES of molecules
molecules=df['reactants'].tolist()
target = 'CCCO'

In [ ]:
obj = SimilarityFP()
obj.FP_score(molecules,target,r'C:\Users\ks\Desktop\ReactionNetwork\FP_example.csv')